# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [18]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'tagsets'])

import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sqlalchemy import create_engine
from sklearn.multioutput import MultiOutputClassifier

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ylc.mariman.MVGM\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ylc.mariman.MVGM\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\ylc.mariman.MVGM\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package tagsets to
[nltk_data]     C:\Users\ylc.mariman.MVGM\AppData\Roaming\nltk_data...
[nltk_data]   Package tagsets is already up-to-date!


In [2]:
# load data from database
def load_data():
    engine = create_engine('sqlite:///../data/DisasterResponse.db')
    sql_query = "SELECT * FROM DisasterTweets"
    df = pd.read_sql(sql_query, engine)
    X = df.message
    y = df[['related', 'request', 'offer',
           'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
           'security', 'military', 'child_alone', 'water', 'food', 'shelter',
           'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
           'infrastructure_related', 'transport', 'buildings', 'electricity',
           'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
           'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
           'other_weather', 'direct_report']]
    return X, y

### 2. Write a tokenization function to process your text data

In [3]:
#wordnet pos_tags can be found here: nltk.help.upenn_tagset()
def tokenize(text):
    """ """ 
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    text = re.sub(url_regex, 'urlplaceholder', text)

    # tokenize
    text = re.sub(r"[^A-Za-z]", " ", text.lower())
    tokens = text.split(" ")
    
    # nltk stopwords + urlplaceholder
    stopwords_new = stopwords.words('english')
    tokens = [word for word in tokens if word not in stopwords.words('english')+ ['urlplaceholder']]
    
    tokens_tagged = nltk.pos_tag(tokens)
    words = [word for word, tag in tokens_tagged if tag in ["JJ", "JJR", "JJS", # Adjectives
                                                            "VB", "VBD", "VBG", "VBN", "VBP", "VBZ", # Verbs
                                                            "NN", "NNP", "NNPS", "NNS", #  Nouns
                                                            "RB", "RBR", "RBS", # Adverbs
                                                            ]]
    
    clean_tokens = [WordNetLemmatizer().lemmatize(w, pos="v") for w in words] # v for verbs
    return clean_tokens

# X, y = load_data()
# test = X[0]
# print(test, tokenize(test))

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [56]:
def build_model():
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X, y = load_data()
X_train, X_test, y_train, y_test = train_test_split(X.values, y)
pipeline = build_model()
pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [51]:
def show_results(y_test, y_pred):
    report = pd.DataFrame(data=[], index=['precision','recall','f1'], columns=y_test.columns)    

    for col in y_test.columns:
        test_col = y_test[col].reset_index(drop=True).rename("test")
        pred_col = pd.DataFrame(y_pred, columns=y_test.columns)[col].rename("predict")
        df_scores = pd.DataFrame([pd.to_numeric(test_col), pd.to_numeric(pred_col)]).T
        tp = np.where((df_scores.test==1) & (df_scores.predict==1), 1, 0).sum()
        fp = np.where((df_scores.test==0) & (df_scores.predict==1), 1, 0).sum()
        fn = np.where((df_scores.test==1) & (df_scores.predict==0), 1, 0).sum()
        
        if tp==0:
            precision=0
            recall=0
            f1=0
        else:
            precision = tp / ( tp + fp)
            recall = tp / (tp + fn)
            f1 = 2 * (precision*recall) / (precision + recall)

        report.loc['precision'][col] = precision
        report.loc['recall'][col] = recall
        report.loc['f1'][col] = f1
    display(report)
    return report

In [55]:
show_results(y_test=y_test, y_pred=y_pred);

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
precision,0.830417,0.802038,0,0.759619,0.650794,0.62069,0.6,0,0.736842,0,...,0,0,0.840696,0.9,0.780423,0,0.869485,0.8125,0.5,0.759516
recall,0.957232,0.500909,0,0.681009,0.078244,0.056604,0.03352,0,0.060345,0,...,0,0,0.697003,0.48423,0.506873,0,0.783113,0.089041,0.023599,0.346761
f1,0.889326,0.616676,0,0.718169,0.139693,0.103746,0.063492,0,0.111554,0,...,0,0,0.762136,0.629674,0.614583,0,0.824042,0.160494,0.04507,0.476139


### 6. Improve your model
Use grid search to find better parameters. 

In [79]:
pipeline = build_model()

parameters = {
    'vect__max_df': [0.5, 1.0, 1.5]
#     'estimator__vect__max_df': [0.5, 1.0, 1.5],
#     'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
#     'features__text_pipeline__vect__max_features': (None, 5000, 10000),
#     'features__text_pipeline__tfidf__use_idf': (True, False),
#     'clf__n_estimators': [50, 100, 200],
    # 'clf__min_samples_split': [2, 3, 4],
    # 'features__transformer_weights': (
    #     {'text_pipeline': 1, 'starting_verb': 0.5},
    #     {'text_pipeline': 0.5, 'starting_verb': 1},
    #     {'text_pipeline': 0.8, 'starting_verb': 1},
    # )
}

cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [82]:
# pipeline.get_params()

In [81]:
cv.fit(X_train, y_train)

y_pred = cv.predict(X_test)

C:\Users\ylc.mariman.MVGM\Miniconda3\envs\udacity\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
5 fits failed out of a total of 15.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\ylc.mariman.MVGM\Miniconda3\envs\udacity\lib\site-packages\sklearn\model_selection\_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\ylc.mariman.MVGM\Miniconda3\envs\udacity\lib\site-packages\sklearn\pipeline.py", line 390, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "C:\Users\ylc.mariman.MVGM\Miniconda3\envs\udacity\lib\site-packages\sklearn\pipeline.py", line

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
import pickle

with open('filename.pickle', 'wb') as handle:
    pickle.dump(cv, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('filename.pickle', 'rb') as handle:
    cv = pickle.load(handle)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.